## Data Preparation

### Datasets

I will be using [fix-yahoo-finance](https://pypi.org/project/fix-yahoo-finance/) python package to get the stock data. Below are the variables of the time series data of a single stock.  
Solution shall be able to get the historical data given a Ticker symbol of the desired stock. 
I will be using data for following tickers from `START_DATE = '2000-01-01'` till current date.

1. Apple(AAPL)
2. Microsoft(MSFT)
3. Amazon(AMZN)
4. IBM(IBM) 
 
### Inputs

The datasets we get from fix-yahoo-finance has below columns. This is a time series daily data, **One record for one day**
Date is the index for the dataset. Ex: 2000-01-01

**1. Open**       - The opeing price of the stock for given day.     
**2. High**       - Maximum price of the stock for given day.   
**3. Low**        - Minimum price of the stock for given day.  
**4. Close**      - Price of the stock at which day was closed for market.  
**5. Adj Close**  - An adjusted closing price is a stock's closing price on any given day of trading that has been amended to                         include any distributions and corporate actions that occurred at any time before the next day's open. 
                    **This will be our Target variable in prediction**  
**6. Volume**     - This value is the indicator of total transactions of the stock market buys/sells for given ticker.

I have used 'Adj Close' as my target variable. And for the features i will using techinical indicators which are derrived from above mentioned inputs. 

### Storing Data

We are using **fix-yahoo-finance** library. To avoid frequent query to this library database and downloads I am saving data locally in a csv file. I am using 'ALLOWED_DELTA' constant to see if there is any need to get latest data. if the time delta is greater than this constant  then only I pull data from fix-yahoo-finance else I use pre-stored data from csv file.

`# Max allowed time delta
ALLOWED_DELTA = datetime.timedelta(days=7) `

All csv file are stored in `data` folder

In [1]:
# Import ML libs
import math
import numpy as np
import pandas as pd
from math import sqrt
from numpy import split
from numpy import array
from numpy import array
from numpy import argmax

# Import required libraries for visualization and printing
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (18, 12)
import matplotlib.dates as mdates
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Import utility for time manupulation
import datetime
import time

#Import other libs
import os
from pathlib import Path

# Import libs for stock market datasets
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf

# One hot encoding for dslo
from keras.utils import to_categorical

# Max allowed time delta
ALLOWED_DELTA = datetime.timedelta(days=7) 
START_DATE = '2000-01-01'

Using TensorFlow backend.


**Helper functions**

In [2]:
def get_date():
    """ Return date in yyyy-mm-dd format """
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d")

def save_data(df,fileName,saveIndex = True):
    """ Saves data to csv file, file name is Ticker symbol """
    df.to_csv("data\\" + fileName, index = saveIndex)
    
def load_data(fileName):
    """ Load data from csv file, file name is Ticker symbol """
    return pd.read_csv("data\\" + fileName, header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])
    
def data_exists(fileName):
    """ Check if file exists in data"""
    my_file = Path("data\\" + fileName )
    res = my_file.is_file()
    return res
    
def get_data(symbol, start_date=START_DATE):
    """ Download and return stock data """
    download_data = True
    if data_exists(symbol):
        # file exists
        data = load_data(symbol)
        #if date is current do not download data
        c_date = data.iloc[-1,0]
        if pd.to_datetime(get_date()) - pd.to_datetime(c_date) < ALLOWED_DELTA:
            download_data = False
    if download_data :  
        try:
            yf.pdr_override()
            # download dataframe
            data = pdr.get_data_yahoo(symbol, start=start_date)
        except ValueError:
            pass
        if Path(symbol).is_file():
            os.remove(symbol)
        save_data(data,symbol)
    return load_data(symbol)

## Impute Data

I am using `fill forward` and then `fill backward` method to replace the `nan` in the data.

In [3]:
def fill_missing_values(df_data):
    """Fill missing values in data frame, in place."""
    df_data.fillna(method='ffill',inplace=True)  
    df_data.fillna(method='bfill',inplace=True)
    return df_data

## Feature Exploration

Finding a better feature for Stock Market prediction is a topic for more exploration and research. I found more I explore more differnt indicators I find. It is only when I apply them in model, I would get better idea about whether they are helping the model learn or making it worst.

I am considering to use 5 features, shifted version **Adj Close**  (Shift depends on the `lookback` factor ) and 3 technical indicators and one functional indicator.

- MACD (Trend)
- MOMETM (Momentum)
- Average True Range (Volume)
- Days since market open (Functional Indicator) It is the days since market is open. This might affect the prices if market was closed for multiple days. My gut feeling tells me this might come handy in predicting target.

**Exponential Moving Average:** An exponential moving average (EMA) is a type of moving average (MA) that places a greater weight and significance on the most recent data points.

`Initial SMA: 10-period sum / 10 `  
`Multiplier: (2 / (Time periods + 1) ) = (2 / (10 + 1) ) = 0.1818 (18.18%)`  
`EMA: {Close - EMA(previous day)} x multiplier + EMA(previous day).`  

https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:moving_averages

**MACD:**  The Moving Average Convergence Divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. The MACD is calculated by subtracting the 26-period Exponential Moving Average (EMA) from the 12-period EMA. 

https://www.investopedia.com/terms/m/macd.asp


**Stochastics oscillator:** The stochastic oscillator is a momentum indicator comparing the closing price of a security to the range of its prices over a certain period of time. The sensitivity of the oscillator to market movements is reducible by adjusting that time period or by taking a moving average of the result.

https://www.investopedia.com/terms/s/stochasticoscillator.asp

**Average True Range:** Is an indicator to measure the volalitility (NOT price direction). The largest of:
- Method A: Current High less the current Low
- Method B: Current High less the previous Close (absolute value)
- Method C: Current Low less the previous Close (absolute value)

https://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:average_true_range_atr

Calculation:
<img src="http://i68.tinypic.com/e0kggi.png">

In [4]:
def MACD(df,period1,period2,periodSignal):
    EMA1 = pd.DataFrame.ewm(df,span=period1).mean()
    EMA2 = pd.DataFrame.ewm(df,span=period2).mean()
    MACD = EMA1-EMA2
    
    Signal = pd.DataFrame.ewm(MACD,periodSignal).mean()
    
    Histogram = MACD-Signal
    
    return Histogram

def stochastics_oscillator(df,period):
    l, h = pd.DataFrame.rolling(df, period).min(), pd.DataFrame.rolling(df, period).max()
    k = 100 * (df - l) / (h - l)
    return k

def ATR(df,period):
    '''
    Method A: Current High less the current Low
    '''
    df['H-L'] = abs(df['High']-df['Low'])
    df['H-PC'] = abs(df['High']-df['Close'].shift(1))
    df['L-PC'] = abs(df['Low']-df['Close'].shift(1))
    TR = df[['H-L','H-PC','L-PC']].max(axis=1)
    return TR.to_frame()

def DaySinceLastOpen(df):
    """This function assumes that timeseries df indexed by date is passed """
    DSLO = []
    DSLO.append(1.0)    
    for i in range(df.shape[0]-1):
        diff = pd.to_datetime(df.index[i+1]) - pd.to_datetime(df.index[i])
        DSLO.append(float(diff.days))
    return pd.Series(DSLO, index=df.index)
        
                                   

In [5]:
tickers=["IBM","AAPL","MSFT","AMZN"]
use_ticker = tickers[1]
df = get_data(use_ticker)

# Impute
df = fill_missing_values(df)

# ease of calculations
dfPrices = df[['Adj Close']]
print(df.head(5))

[*********************100%***********************]  1 of 1 downloaded
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2000-01-03  3.745536  4.017857  3.631696  3.997768   2.677157  133949200
2000-01-04  3.866071  3.950893  3.613839  3.660714   2.451444  128094400
2000-01-05  3.705357  3.948661  3.678571  3.714286   2.487319  194580400
2000-01-06  3.790179  3.821429  3.392857  3.392857   2.272070  191993200
2000-01-07  3.446429  3.607143  3.410714  3.553571   2.379695  115183600


## Derrived Feature Calculations 

In [10]:
macd = MACD(dfPrices,12,26,9)
macd.rename(columns={'Adj Close':'MACD'}, inplace=True)

In [11]:
stochastics = stochastics_oscillator(dfPrices,14)
stochastics.rename(columns={'Adj Close':'MOMETM'}, inplace=True)

In [12]:
atr = ATR(df,14)
atr.rename(columns={0:'ATR'}, inplace=True)

**DSLO** :   
    Since `dslo` is a categorical variable we need to apply one hot encoding on this variable before we use this as a feature.

In [13]:
# get categorical dslo
dslo = DaySinceLastOpen(df) 
# define example
data = array(dslo)
# one hot encode
encoded = to_categorical(data)
encoded_dslo = pd.DataFrame(encoded,index= df.index)
encoded_dslo.rename(columns={0:'dslo_0',1:'dslo_1',2:'dslo_2',3:'dslo_3',4:'dslo_4',5:'dslo_5',6:'dslo_6',7:'dslo_7'}, inplace=True)

## Save final data

In [14]:
final_data = pd.concat([dfPrices,macd,stochastics,atr,encoded_dslo], axis=1)
# Delete the entries with missing values (where the stochastics couldn't be computed yet) because have a lot of datapoints ;)
final_data = final_data.dropna()

In [15]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4774 entries, 2000-01-21 to 2019-01-11
Data columns (total 12 columns):
Adj Close    4774 non-null float64
MACD         4774 non-null float64
MOMETM       4774 non-null float64
ATR          4774 non-null float64
dslo_0       4774 non-null float32
dslo_1       4774 non-null float32
dslo_2       4774 non-null float32
dslo_3       4774 non-null float32
dslo_4       4774 non-null float32
dslo_5       4774 non-null float32
dslo_6       4774 non-null float32
dslo_7       4774 non-null float32
dtypes: float32(8), float64(4)
memory usage: 335.7 KB


In [16]:
final_data.head(10)

,Adj Close,MACD,MOMETM,ATR,dslo_0,dslo_1,dslo_2,dslo_3,dslo_4,dslo_5,dslo_6,dslo_7
Date,,,,,,,,,,,,
2000-01-21,2.662209,0.035119,91.686530,0.145089,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-24,2.541132,0.030618,72.446703,0.272322,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2000-01-25,2.684630,0.034074,95.249355,0.383929,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-26,2.635303,0.032370,87.411013,0.158482,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-27,2.630817,0.029670,86.698162,0.214285,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-28,2.430518,0.015247,54.869507,0.366071,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-01-31,2.481340,0.007609,62.945413,0.334821,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2000-02-01,2.397632,-0.003005,49.643733,0.178571,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-02,2.363252,-0.012446,12.313468,0.183035,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
final_data.describe()

,Adj Close,MACD,MOMETM,ATR,dslo_0,dslo_1,dslo_2,dslo_3,dslo_4,dslo_5,dslo_6,dslo_7
count,4774.000000,4774.000000,4774.000000,4774.000000,4774.0,4774.000000,4774.000000,4774.000000,4774.000000,4774.000000,4774.0,4774.000000
mean,45.514683,-0.015970,58.120048,1.125256,0.0,0.782782,0.009845,0.181190,0.025555,0.000419,0.0,0.000209
std,55.174150,0.604943,36.825707,1.435248,0.0,0.412390,0.098742,0.385223,0.157824,0.020465,0.0,0.014474
min,0.627570,-4.903227,0.000000,0.013571,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,2.989566,-0.107592,22.756815,0.165714,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,17.662357,0.002236,65.717552,0.692143,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,69.123907,0.114137,95.694468,1.521427,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,231.263092,3.484961,100.000000,16.800003,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


In [18]:
save_data(final_data,use_ticker+"_final")